In [1]:
from xclim import ensembles
import xarray as xr
import os

## Create ensemble files from running CMIP6_light.py
This script will loop over all files found in a speficic folder `lightpath`and find files of teh same variable and scenario to create ensemble files from. The various variables to create ensembles for include `["uvb_","uv_","uvi_","par_", "uvb_"]`.

The output is stored under folder `ensemble_path` and the results are used to create the final plots of modeled lightlyfor the paper using notebooks `CMIP6_plot_light_results.ipynb`.


In [2]:
# Define what models, scenarios, and variables to calculate ensemble files from
models=["CanESM5","CMCC-ESM2","MPI-ESM1-2-HR", "ACCESS-ESM1-5", "MPI-ESM1-2-LR"]
create_ensemble = True
scenarios=["ssp245","ssp585"]
var_names=["par_","uvb_","uv_","uvi_", "uvb_"] #,"_tos.nc","_siconc.nc"]
var_names=["uvb_","uv_"]

In [ ]:

def create_ensemble_files(scenarios, var_names):

    lightpath="../oceanography/light/ncfiles/"
    ensemble_path="../oceanography/light/ncfiles/ensemble/"
    if not os.path.exists(ensemble_path):
        os.makedirs(ensemble_path, exist_ok=True)

    for var_name in var_names:

        for scenario in scenarios:
            ds_list=[]
            if var_name not in ["_tos.nc", "_siconc.nc"]:
                ensemble_stats=ensemble_path+"{}ensemble_stats_{}.nc".format(var_name, scenario)
                ensemble_perc=ensemble_path+"{}ensemble_perc_{}.nc".format(var_name, scenario)
            else:
                if var_name=="_tos.nc":
                    ensemble_stats=ensemble_path+"tos_ensemble_stats_{}.nc".format(scenario)
                    ensemble_perc=ensemble_path+"tos_ensemble_perc_{}.nc".format(scenario)
                elif var_name=="_siconc.nc":
                    ensemble_stats=ensemble_path+"siconc_ensemble_{}.nc".format(scenario)
                    ensemble_perc=ensemble_path+"siconc_ensemble_perc_{}.nc".format(scenario)

            if os.path.exists(ensemble_stats):
                os.remove(ensemble_stats)
            if os.path.exists(ensemble_perc):
                os.remove(ensemble_perc)

            for f in os.listdir(lightpath):

                if ((f.startswith(var_name) and scenario in f and "ensemble" not in f) or
                    (f.endswith(var_name) and scenario in f and "ensemble" not in f)):
                    if var_name not in ["uv_srf"] and not f.startswith("uv_srf"):
                        print("{} - {}: found {}".format(var_name, scenario, f))

                        # Filter to only use the models we prefer (UKESM1-0-LL excluded)
                        if any(model in f for model in models):
                            ds=xr.open_dataset(lightpath+f)
                            ds_list.append(ds)

            print("Creating ensemble for {} and scenario {}".format(var_name, scenario))
            ens = ensembles.create_ensemble(ds_list) #.load()
            ens_stats = ensembles.ensemble_mean_std_max_min(ens)
            ens_perc = ensembles.ensemble_percentiles(ens, values=[2.5, 5, 7.5, 15, 50, 85, 90, 97.5], split=False)

            ens_stats.to_netcdf(ensemble_stats)
            ens_perc.to_netcdf(ensemble_perc)

            print("Created ensemble {}".format(ensemble_stats))

create_ensemble_files(scenarios, var_names)

uvb_ - ssp245: found uvb_CMCC-ESM2_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
uvb_ - ssp245: found uvb_MPI-ESM1-2-HR_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
uvb_ - ssp245: found uvb_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
uvb_ - ssp245: found uvb_MPI-ESM1-2-LR_r2i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
uvb_ - ssp245: found uvb_ACCESS-ESM1-5_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Creating ensemble for uvb_ and scenario ssp245


/Users/trondkr/opt/anaconda3/envs/light/lib/python3.9/site-packages/xclim/core/utils.py:412: RuntimeWarning: All-NaN slice encountered
  result = np.where(np.isnan(interpolation), np.nanmax(arr, axis=0), interpolation)


Created ensemble ../oceanography/light/ncfiles/ensemble/uvb_ensemble_stats_ssp245.nc
uvb_ - ssp585: found uvb_MPI-ESM1-2-LR_r2i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
uvb_ - ssp585: found uvb_MPI-ESM1-2-LR_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
uvb_ - ssp585: found uvb_ACCESS-ESM1-5_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
uvb_ - ssp585: found uvb_CMCC-ESM2_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
uvb_ - ssp585: found uvb_UKESM1-0-LL_r1i1p1f2_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
uvb_ - ssp585: found uvb_MPI-ESM1-2-HR_r1i1p1f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
uvb_ - ssp585: found uvb_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Creating ensemble for uvb_ and scenario ssp585
